**Necessary packages to make this work**

for the script
- conda install -c open3d-admin open3d==0.9.0
-  conda install -c anaconda scipy 
-  conda install -c anaconda pandas 
-  conda install -c anaconda networkx 
-  conda install -c conda-forge matplotlib 
-  conda install -c obob pymatreader 
-  conda install -c anaconda numpy 
-  conda install -c conda-forge opencv 
-  pip install imageio #use pip here to avoid conflict 
- conda install -c conda-forge jupyterlab
- pip install pycpd
- pip install cython
- git clone https://github.com/gattia/cycpd
- cd cycpd
- sudo python setup.py install
<!-- - conda install -c anaconda ipykernel -->

*For nice display*
- conda install -c conda-forge ipympl
- conda install -c conda-forge nodejs
- jupyter labextension install @jupyter-widgets/jupyterlab-manager
- jupyter lab build


for nice display in jupyter lab



In [1]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes

**First load the dates of interest**

This is some non very interesting code to extract all the folder names and the associated dates in a python understandable way

In [2]:
plate = 13
directory='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE' 
listdir=os.listdir(directory) #This line will run only if the vpn is connected to amolf, if by the time you're trying this you still don't have access to amolf system
#manually find a way to find the dates you're interested in
#You can use the code below for plate 13 (to uncomment)
# plate = 13
# date_init = datetime(2020,7,1,19,57)
# dates_datetime = [date_init+timedelta(hours=4)*i for i in range(24)]
# dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:5] #out of all the folders that relate to the plate of interest, we extract only a subset, here the 4 first ones.
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]

**The experiment objects**

An experiment is associated with a plate so we create the Experiment.

Then we only need to load the date that we are interested in.


In [3]:
exp= Experiment(plate) #Create the object
exp.load(dates,local=False) #load the dates, if local is set to True, no connection to Amolf system is needed (and loading will be faster)
#In that cas data should all put in a 'Data' folder in the directory of the notebook and the code with the original names. (see source code)


To plot the skeleton of the experiment at any timestep. One can use the **plot** method., the first parameter is the list of time-step (index of the corresponding dates that we want to plot) while the second parameter is the list of list of nodes that we want to plot at each of these timesteps (see later).

In [16]:
exp.plot([2],[[]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The fancy display is quite heavy to handle. I recommend regularly to use **plt.close('all')** to avoid memory overload. This will close the interactive display of **exp.plot**.

In [9]:
plt.close('all')

An experiment object has different attributes, some of them we actually don't access that often but I mention them so you know they are there.

In [7]:
print(type(exp.nx_graph),type(exp.nx_graph[0])) #The graphs at each time step, so nx_graph[0] is the graph at timestep 0 (corresponding to dates[0])
print(type(exp.positions),type(exp.positions[0])) #This is not super handy, but the positions are separated from the graph

<class 'list'> <class 'networkx.classes.graph.Graph'>
<class 'list'> <class 'dict'>


So the best way to actually walk through the graph is through nodes, hyphae and edges.

In [17]:
nodes = exp.nodes #the list of all nodes that have been identified in the network

Nodes are abstract objects that correspond to node that have been identified across timesteps. They have different methods to navigate through them.

In [18]:
node = choice(nodes) #choose one random node for demo.
t = 2 
#first one can ask if the node is present in the network at a certain time t
print(node.is_in(t))
if node.is_in(t):
    print(node.degree(t)) #one can get the degree of this node at the timestep t (this degree can change over time due to anastomosis)
    print(node.neighbours(t)) #one can get the neighbours of the node at time t, this is also time dependant
    print("here are the neighbouring nodes")
    for node in node.neighbours(t):
        print(f'label={node.label}, degree = {node.degree(t)}, position = {node.pos(t)}')
#One can also go through the edges stemming from a node
    print("here are the edges from this node")
    for edge in node.edges(t):
        print(f'edge = {edge}', f'begining = {edge.begin}, end = {edge.end}') #when we print an edge we see the begining of the edge and it's end
        #edges are also abstract object, at a certain time t, one can access their pixel list, simple functions based on this pixel list have been implemented.
        #the pixel list is the list of pixel from begin to end
        print(f'first_pixel = {edge.pixel_list(t)[0]} \n to compare with {edge.begin.pos(t)}')
        print(f'orientation =  {edge.orientation_whole(t)}')
        

True
1
[Node(2784)]
here are the neighbouring nodes
label=2784, degree = 3, position = [14503 22443]
here are the edges from this node
edge = (Node(2784), Node(2805)) begining = 2784, end = 2805
first_pixel = (14503, 22443) 
 to compare with [14503 22443]
orientation =  144.10733431225444
edge = (Node(2784), Node(397)) begining = 2784, end = 397
first_pixel = (14503, 22443) 
 to compare with [14503 22443]
orientation =  31.865977693603682
edge = (Node(2784), Node(2789)) begining = 2784, end = 2789
first_pixel = (14503, 22443) 
 to compare with [14503 22443]
orientation =  -86.42366562500266


to see how this all looks like, let's plot all these nodes!
You'll notice that the coordinates as ther are shown on the axis don't correspond to the ones above, this is because we compress the image to be able to display it.

In [15]:
exp.plot([0,1,2,3],[[node.label]+[neighbour.label for neighbour in node.neighbours(t)] for t in range(len(dates))])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Coming soon : hyphaes 